In [40]:
import pandas as pd

In [41]:
def dfIsComplete(df):
    odf = df.copy()
    dff = df.groupby(['date', 'state']) \
        .sum() \
        .unstack(fill_value=0) \
        .asfreq('D', fill_value=0) \
        .stack() \
        .sort_index(level=1) \
        .reset_index() \
        .sort_values(by=['date', 'state'])
    dff.date = dff.date.dt.date

    assert len(odf) == len(dff)

In [42]:

icu_cases_path =  'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/icu.csv'
vax_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/vaccination/vax_malaysia.csv'
death_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/deaths_malaysia.csv'
cases_malaysia_path = 'https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv'

In [43]:
beginningDate = '2021-01-01'
endingDate = '2021-06-30'

In [44]:
icu_cases = pd.read_csv(icu_cases_path, usecols= ['date', 'state', 'beds_icu_covid'], parse_dates= ['date'])
vax_malaysia = pd.read_csv(vax_malaysia_path)
death_malaysia = pd.read_csv(death_malaysia_path)
cases_malaysia = pd.read_csv(cases_malaysia_path)

icu_cases['date'] = pd.to_datetime(icu_cases['date'])
vax_malaysia['date'] = pd.to_datetime(vax_malaysia['date'])

icu_cases = icu_cases[icu_cases['date'] >= beginningDate]
icu_cases = icu_cases[icu_cases['date'] <= endingDate]

vax_malaysia = vax_malaysia[vax_malaysia['date'] >= beginningDate]
vax_malaysia = vax_malaysia[vax_malaysia['date'] <= endingDate]

death_malaysia = death_malaysia[death_malaysia['date'] >= beginningDate]
death_malaysia = death_malaysia[death_malaysia['date'] <= endingDate]

cases_malaysia = cases_malaysia[cases_malaysia['date'] >= beginningDate]
cases_malaysia = cases_malaysia[cases_malaysia['date'] <= endingDate]
#print(vax_malaysia.head(5))

In [56]:
casesAgesChild = cases_malaysia.iloc[:, 10]
casesAgesAdole = cases_malaysia.iloc[:, 11]
casesAgesAdult = cases_malaysia.iloc[:, 12]
casesAgesElder = cases_malaysia.iloc[:, 13]
casesNum = cases_malaysia.iloc[:, 1]
casesUnvax = cases_malaysia.iloc[:, 6]
casesPvax = cases_malaysia.iloc[:, 7]
casesFvax = cases_malaysia.iloc[:, 8]

icuCasesTotal = icu_cases.groupby('date')['beds_icu_covid'].sum().reset_index()

deathNum = death_malaysia.iloc[:, 1]
deathUnvax = death_malaysia.iloc[:, 5]
deathPvax = death_malaysia.iloc[:, 6]
deathFvax = death_malaysia.iloc[:, 7]


print(icuCasesTotal)


          date  beds_icu_covid
0   2021-01-01             466
1   2021-01-02             466
2   2021-01-03             466
3   2021-01-04             466
4   2021-01-05             466
..         ...             ...
176 2021-06-26            1207
177 2021-06-27            1245
178 2021-06-28            1250
179 2021-06-29            1250
180 2021-06-30            1252

[181 rows x 2 columns]
